In [4]:
#Завдання 1
def is_on_curve(x, y, a, b, p):
    """
    перевірка, чи точка (x, y) належить кривій y^2 = x^3 + ax + b (mod p)
    """
    return (y ** 2) % p == (x ** 3 + a * x + b) % p

def inverse_mod(k, p):
    """
    обчислення оберненого елемента k за модулем p
    """
    return pow(k, -1, p)  # використовуємо функцію pow для оберненого елемента

def add_points(p1, p2, a, p):
    """
    додавання двох точок p1 і p2 на еліптичній кривій y^2 = x^3 + ax + b (mod p)
    """
    x1, y1 = p1
    x2, y2 = p2

    if p1 == p2:  # обчислення для подвоєння точки
        lambd = (3 * x1 ** 2 + a) * inverse_mod(2 * y1, p) % p
    else:  # обчислення для додавання двох різних точок
        lambd = (y2 - y1) * inverse_mod(x2 - x1, p) % p

    x3 = (lambd ** 2 - x1 - x2) % p
    y3 = (lambd * (x1 - x3) - y1) % p

    return x3, y3

def main():
    # параметри еліптичної кривої y^2 = x^3 - x + 3 mod 127
    a = -1  # коефіцієнт при x
    b = 3   # вільний член
    p = 127 # просте число, модуль

    # точки P і Q
    P = (62, 12)
    Q = (66, 63)

    # перевірка належності точок кривій
    print(f"точка P{P} належить кривій:", is_on_curve(P[0], P[1], a, b, p))
    print(f"точка Q{Q} належить кривій:", is_on_curve(Q[0], Q[1], a, b, p))

    # якщо точки належать кривій, обчислюємо P + Q
    if is_on_curve(P[0], P[1], a, b, p) and is_on_curve(Q[0], Q[1], a, b, p):
        R = add_points(P, Q, a, p)
        print(f"сума P + Q = {R}")
    else:
        print("одна або обидві точки не належать кривій")

if __name__ == "__main__":
    main()


точка P(62, 12) належить кривій: True
точка Q(66, 63) належить кривій: True
сума P + Q = (106, 62)


In [2]:
#Завдання 2

from sympy import symbols, mod_inverse

# параметри еліптичної кривої
a, b, p = -1, 3, 127

def is_point_on_curve(x, y):
    """Перевіряє, чи належить точка еліптичній кривій."""
    return (y ** 2) % p == (x ** 3 + a * x + b) % p

def point_addition(p1, p2):
    """Додавання точок на еліптичній кривій."""
    if p1 == (None, None):
        return p2
    if p2 == (None, None):
        return p1

    x1, y1 = p1
    x2, y2 = p2

    if x1 == x2 and y1 == -y2 % p:
        return (None, None)

    if p1 != p2:
        m = (y2 - y1) * mod_inverse(x2 - x1, p) % p
    else:
        m = (3 * x1 ** 2 + a) * mod_inverse(2 * y1, p) % p

    x3 = (m ** 2 - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p

    return (x3, y3)

def scalar_multiplication(k, point):
    """Множення точки на скаляр."""
    result = (None, None)  # нейтральний елемент
    addend = point

    while k:
        if k & 1:
            result = point_addition(result, addend)
        addend = point_addition(addend, addend)
        k >>= 1

    return result

# генератор Q та перевірка, що він на кривій
q = (66, 63)
assert is_point_on_curve(*q), "Точка Q не лежить на кривій."

# приватний ключ та відкритий ключ
private_key = 35  # обираємо приватний ключ
g = q
public_key = scalar_multiplication(private_key, g)

print("Приватний ключ:", private_key)
print("Відкритий ключ:", public_key)

# шифрування точки P
p_point = (62, 12)
assert is_point_on_curve(*p_point), "Точка P не лежить на кривій."

k = 17  # випадкове ціле число для шифрування
c1 = scalar_multiplication(k, g)
c2 = point_addition(p_point, scalar_multiplication(k, public_key))

print("Зашифроване повідомлення (C1, C2):", c1, c2)

# розшифрування
s = scalar_multiplication(private_key, c1)
s_inverse = (s[0], -s[1] % p)  # обернена точка до S
original_point = point_addition(c2, s_inverse)

print("Розшифроване повідомлення:", original_point)


Приватний ключ: 35
Відкритий ключ: (17, 33)
Зашифроване повідомлення (C1, C2): (68, 13) (80, 42)
Розшифроване повідомлення: (62, 12)
